<a href="https://colab.research.google.com/github/steliosg23/Data_Challenge_2025/blob/main/3_dataset_saving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
import numpy as np
import torch.nn as nn
from tqdm import tqdm
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Set the base directory (adjust this path to your directory)
base_dir = '/content/drive/MyDrive/Data Science AUEB/Data Challenge/data/'

# Load data
data_path = f'{base_dir}expanded_df.csv'  # Replace this with the actual CSV file path
df = pd.read_csv(data_path)

# Preprocess the data
df['text'] = df['title_and_description']  # Combining title and description for BERT input
df = df[['product_id','text', 'price','neighbor_product_id','class_label']]

# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load test dataset
test_path = '/content/drive/MyDrive/Data Science AUEB/Data Challenge/data/test.txt'
test_df = pd.read_csv(test_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# Convert to Dataset
dataset = Dataset.from_pandas(df)

# Tokenize the dataset using multiple processes for parallelism
def tokenize_with_progress(dataset, tokenizer_function, num_proc=100):  # num_proc for parallelism
    # Manually wrap the map function with tqdm for progress tracking
    tokenized_data = []
    for example in tqdm(dataset, desc="Tokenizing dataset", total=len(dataset)):
        tokenized_data.append(tokenizer_function(example))
    return tokenized_data

# Tokenize the dataset using multiple processes
tokenized_datasets = tokenize_with_progress(dataset, tokenize_function, num_proc=100)



In [ ]:
# prompt: extract tokenized_datasets in pkl and save in same directory

import pickle

# Save tokenized_datasets to a pickle file in the same directory
with open(f'{base_dir}tokenized_datasets.pkl', 'wb') as f:
    pickle.dump(tokenized_datasets, f)


In [ ]:
# prompt: load the pkl file

import pickle

# Load the tokenized dataset from the pickle file
with open(f'{base_dir}tokenized_datasets.pkl', "rb") as f:
  tokenized_datasets = pickle.load(f)


In [ ]:
# Convert to Dataset
dataset = Dataset.from_pandas(df)
# Update the dataset with tokenized data
dataset = dataset.add_column("input_ids", [item["input_ids"] for item in tokenized_datasets])
dataset = dataset.add_column("attention_mask", [item["attention_mask"] for item in tokenized_datasets])




In [ ]:


# Convert the dataset to a Pandas DataFrame
df_dataset = dataset.to_pandas()

# Save the DataFrame to a CSV file
df_dataset.to_csv(f'{base_dir}dataset.csv', index=False)


In [ ]:
# Save tokenized_datasets to a pickle file in the same directory
with open(f'{base_dir}final_dataset.pkl', 'wb') as f:
    pickle.dump(dataset, f)